In [24]:
import theano
import theano.tensor as T
import numpy as np
from collections import OrderedDict

In [82]:
def quantize(input, step_size, n_bits):
    n_steps = 2**(n_bits-1)-1
    temp = T.round(input / step_size)
    temp = T.minimum(temp, n_steps)
    temp = T.maximum(temp, -n_steps)
    return temp*step_size

def get_step_size(s):
    return T.cast(s,'float32')





In [79]:
a = get_step_size(0.1)
a = get_step_size(0.2)
s = np.asarray(0.1, dtype = 'float32')

In [102]:
x = np.asarray(np.random.rand(5,5)-0.5,dtype = 'float32')
s = np.asarray(0.1, dtype = 'float32')
n_bits = 3

class test():
    def __init__(self, nbits):
        self.x_shared = theano.shared(x)
        self.s_shared = theano.shared(s)
        self.fixed_shared = theano.shared(x)
        self.n = nbits
        self.param_fixed = [fixed_shared]
        self.param_quantized = [quantize(param_fixed, s_shared, n_bits)]

        self.y = T.sum(self.x_shared)
        self.fixed_updates = OrderedDict()
        self.updates = OrderedDict()
        
    
    def get_updates(self):
        n_step = T.pow(2,self.n-1)-1
        #self.fixed_updates[self.fixed_shared] = T.maximum(T.minimum(T.round(self.x_shared / self.s_shared), n_step),-n_step)
        self.fixed_updates[self.x_shared] = self.x_shared - [T.grad(self.y, self.x_shared)]
        return fixed_updates
        #self.fixed_updates[x_shared] = T.grad(self.y, self.x_shared)



In [103]:
a = test(3)
upd = a.get_updates()
#test_fun = theano.function(inputs = [],
#                          outputs = [],
#                          updates = upd)
print(upd)

OrderedDict([(<TensorType(float32, matrix)>, Elemwise{mul,no_inplace}.0)])


In [104]:
fun1 = theano.function(inputs = [], updates = fixed_updates, outputs = [])

TypeError: ('An update must have the same type as the original shared variable (shared_var=<TensorType(float32, matrix)>, shared_var.type=TensorType(float32, matrix), update_val=Elemwise{mul,no_inplace}.0, update_val.type=TensorType(float32, (True, False, False))).', 'If the difference is related to the broadcast pattern, you can call the tensor.unbroadcast(var, axis_to_unbroadcast[, ...]) function to remove broadcastable dimensions.')

In [45]:
fun2 = theano.function(inputs = [], outputs = quantize(x_shared, s_shared, 3))

In [48]:
print(x)
print(fun2())
print(type(fun2()))

[[-0.27958101  0.28974539 -0.31195787  0.08672415  0.37770763]
 [-0.06198593 -0.23962581  0.07360829 -0.02703791 -0.44310743]
 [-0.45039165 -0.28791687 -0.44049704 -0.41792256  0.1068239 ]
 [-0.10970719 -0.12472475 -0.49233606  0.25047517 -0.35244679]
 [ 0.41071752  0.00092027 -0.01045024 -0.29728943  0.20029037]]
[[-0.30000001  0.30000001 -0.30000001  0.1         0.30000001]
 [-0.1        -0.2         0.1        -0.         -0.30000001]
 [-0.30000001 -0.30000001 -0.30000001 -0.30000001  0.1       ]
 [-0.1        -0.1        -0.30000001  0.30000001 -0.30000001]
 [ 0.30000001  0.         -0.         -0.30000001  0.2       ]]
<class 'numpy.ndarray'>


In [50]:
T.grad(y,x_shared)

Elemwise{second}.0

In [67]:
fun3 = theano.function(inputs = [], outputs = T.grad(y,x_shared))

In [68]:
fun3()

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]], dtype=float32)

In [65]:
updates = []
gparams = [T.grad(y,x_shared)]
count = 0
for n, (param, grad) in enumerate(zip(param_fixed, gparams)):
    count+=1
    updates.append((param, param - grad))
print(count)
updates = OrderedDict(updates)

print(updates)

1
OrderedDict([(<TensorType(float32, matrix)>, Elemwise{sub,no_inplace}.0)])


In [63]:
fun4 = theano.function(inputs = [], outputs = [], updates = updates)

In [66]:
print(x_shared.get_value(),'\n\n')
fun4()
print(x_shared.get_value())

[[-0.40804988  0.46965244 -0.48871022 -0.22406742  0.46042252]
 [ 0.40825397  0.36835557 -0.25598183  0.32126179  0.2431259 ]
 [-0.06048227  0.3032468  -0.32513893 -0.08931643  0.32282916]
 [ 0.14846344  0.18819812  0.34832835 -0.19162592 -0.04929791]
 [ 0.04577921  0.35213175  0.13132583 -0.16226082 -0.2660819 ]] 


[[-0.40804988  0.46965244 -0.48871022 -0.22406742  0.46042252]
 [ 0.40825397  0.36835557 -0.25598183  0.32126179  0.2431259 ]
 [-0.06048227  0.3032468  -0.32513893 -0.08931643  0.32282916]
 [ 0.14846344  0.18819812  0.34832835 -0.19162592 -0.04929791]
 [ 0.04577921  0.35213175  0.13132583 -0.16226082 -0.2660819 ]]
